In [1]:
import argparse
from datetime import date
import os
import sys
import cv2
import json
import numpy as np
import time
import tensorflow as tf

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD

import matplotlib.pyplot as plt
from model import efficientdet
from utils import preprocess_image, postprocess_boxes
from augmentor.color import VisualEffect
from augmentor.misc import MiscEffect
from losses import smooth_l1, focal, smooth_l1_quad
from efficientnet import BASE_WEIGHTS_PATH, WEIGHTS_HASHES

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(device=gpus[1], enable=True)

In [3]:
phi = 5
weighted_bifpn = False
model_path = '/opt/mxl/Cigarette_Image_OD/checkpoints/2020-07-24/pascal_50_0.0528_0.0809.h5'
image_sizes = (512, 640, 768, 896, 1024, 1280, 1408)
image_size = image_sizes[phi]
classes = ['pack']
num_classes = len(classes)
score_threshold = 0.9
colors = [np.random.randint(255,0,0).tolist() for _ in range(num_classes)]
_, model = efficientdet(phi=phi,
                        weighted_bifpn=weighted_bifpn,
                        num_classes=num_classes,
                        score_threshold=score_threshold)
model.load_weights(model_path, by_name=True)
plt.figure(figsize=(15,9))    

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

In [4]:
def inner_draw_boxes(image, boxes, scores, labels, colors, classes):
    for b, l, s in zip(boxes, labels, scores):
        class_id = int(l)
        class_name = classes[class_id]
    
        xmin, ymin, xmax, ymax = list(map(int, b))
        score = '{:.4f}'.format(s)
        color = colors[class_id]
        label = '-'.join([class_name, score])
    
        ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 5)
        #cv2.rectangle(image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
        #cv2.putText(image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

In [5]:
image_path = '/ai/data/test/'
filenames= os.listdir(image_path)

In [6]:
for filename in filenames:   
    
    image = cv2.imread(image_path + filename)
    src_image = image.copy()
    # BGR -> RGB
    image = image[:, :, ::-1]
    h, w = image.shape[:2]

    image, scale = preprocess_image(image, image_size=image_size)
    # run network
    start = time.time()
    boxes, scores, labels = model.predict_on_batch([np.expand_dims(image, axis=0)])
    boxes, scores, labels = np.squeeze(boxes), np.squeeze(scores), np.squeeze(labels)
    print(filename + ' 预测时间',time.time() - start)
    boxes = postprocess_boxes(boxes=boxes, scale=scale, height=h, width=w)
    # select indices which have a score above the threshold
    indices = np.where(scores[:] > score_threshold)[0]

    # select those detections
    boxes = boxes[indices]
    labels = labels[indices]

    inner_draw_boxes(src_image, boxes, scores, labels, (255,0,0), classes)
    
    cv2.imwrite(filename, src_image)

1.jpg 预测时间 11.401500940322876
2.jpg 预测时间 0.3147695064544678
3.jpg 预测时间 0.3031799793243408
4.jpg 预测时间 0.31380462646484375
5.jpg 预测时间 0.3031017780303955
6.jpg 预测时间 0.3144185543060303
7.jpg 预测时间 0.3033182621002197
8.jpg 预测时间 0.31372499465942383
9.jpg 预测时间 0.3095848560333252
10.jpg 预测时间 0.3202054500579834
11.jpg 预测时间 0.31667399406433105
